In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

import pythainlp
from pythainlp import word_tokenize
from pythainlp.util.normalize import normalize, remove_tonemark
from pythainlp.ulmfit.preprocess import (
    fix_html,
    lowercase_all,
    replace_rep_nonum,
    replace_wrep_post_nonum,
    rm_brackets,
    rm_useless_newlines,
    rm_useless_spaces,
    spec_add_spaces,
    ungroup_emoji,
)

C:\Users\GMGPJ\Anaconda3\envs\sentiment_analysis_test\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Pre-processing "Thai" dataset

In [2]:
# read dataset containing only reviews in Thai
dataset_th_df = pd.read_csv('train_thai_dataset.csv')

In [3]:
dataset_th_df.shape

(39723, 2)

### General Preprocessing before Tokenization

In [4]:
def preprocess_review(review):
    """
    Preprocessing reviews
    """
    
    # remove URLs
    review = re.sub(r'http\S+', '', review)
    review = re.sub(r'www\S+', '', review)
    
    # remove html tags
    review = BeautifulSoup(review).get_text()
    
    # replace newlines with whitespace
    review = review.replace('\n', ' ')
    
    # remove multiple consecutive spaces
    review = re.sub(' +', ' ', review)
    
    # strip() method removes whitespace, at the beginning and end (both sides) of a string
    review = review.strip()
    
    return review

In [5]:
# apply general pre-processing steps 
dataset_th_df['Text'] = dataset_th_df['Text'].apply(preprocess_review)

In [6]:
"""
Text normalization from pythainlp.util.normalize

Normalize and clean Thai text with normalizing rules as follows:
* Remove zero-width spaces
* Remove duplicate spaces
* Reorder tone marks and vowels to standard order/spelling
* Remove duplicate vowels and signs
* Remove duplicate tone marks
* Remove dangling non-base characters at the beginning of text

--> normalizing the character order is required for the machine to understand the seemingly similar tokens

Remove all Thai tone marks from the text from pythainlp.util.remove_tonemark

"""

dataset_th_df['Text'] = dataset_th_df['Text'].apply(normalize)
dataset_th_df['Text'] = dataset_th_df['Text'].apply(remove_tonemark)

In [7]:
"""
Customized Preprocessing before Tokenization from pythainlp.ulmfit.preprocess
"""

# Replace html string in text
dataset_th_df['Text'] = dataset_th_df['Text'].apply(fix_html)

# Replace repetitions at the character level in `text` after the repetition  'น้อยยยยยยยย' --> 'น้อย xxrep '
dataset_th_df['Text'] = dataset_th_df['Text'].apply(replace_rep_nonum)

# Remove all empty brackets and artifacts within brackets from review
dataset_th_df['Text'] = dataset_th_df['Text'].apply(rm_brackets)

# Remove multiple newlines in review
dataset_th_df['Text'] = dataset_th_df['Text'].apply(rm_useless_newlines)

# Remove multiple spaces in review
dataset_th_df['Text'] = dataset_th_df['Text'].apply(rm_useless_spaces)

# Add spaces around / and # in review
dataset_th_df['Text'] = dataset_th_df['Text'].apply(spec_add_spaces)

In [8]:
# In an online environment, Thai people often express laughter in written language with the repetition of ‘5’ 
# there are no more than 3 consecutive ‘5’, otherwise - replaced all with a special token

dataset_th_df[dataset_th_df.apply(lambda x: '5555' in x['Text'], axis=1)].shape

(0, 2)

### Tokenization

In [9]:
# pyThaiNLP - Default word tokenizer (“newmm”) use maximum matching algorithm.
dataset_th_df['Text_token'] = dataset_th_df['Text'].apply(word_tokenize)

# show 5 samples
dataset_th_df.loc[:5, ['Text','Text_token']]

,Text,Text_token
0,นอกจากเป็นรานนำแข็งใสสไตล์เกาหลีแลว ยังมีเมนูเ...,"[นอกจาก, เป็น, ราน, นำ, แข็ง, ใส, สไตล์, เกาหล..."
1,มือกลางวันวันนี เราแวะมาทานที Jewelry Trade Ce...,"[มือ, กลางวัน, วัน, นี, , เรา, แวะ, มา, ทา, น..."
2,รานเล็กๆบนถนนนิมมาน ตังอยุปากซอยทางจะเขาไปประเ...,"[ราน, เล็ก, ๆบน, ถนน, นิมมาน, , ตัง, อ, ยุ, ป..."
3,แถวๆ เขือนปาสักฯ นันก็มีรานอาหารอยูหลายรานเลย ...,"[แถวๆ, , เขือ, น, ปา, สัก, ฯ, , นัน, ก็, มี,..."
4,เป็นรานอาหารชายทะเล มีอาหารทะเลสดๆใหกินทุกวัน ...,"[เป็น, ราน, อาหาร, ชายทะเล, , มี, อาหารทะเล, ..."
5,รานนีคอนขางจะเป็นรานทีมีชือเสียง ประมาณวาแขกไป...,"[ราน, นี, คอน, ขาง, จะ, เป็น, รา, นที, มี, ชือ..."


### General Preprocessing after Tokenization

General preprocessing techniques after tokenization were used. This includes ungrouping the emoji’s from text, and to lowercase all English words.

In [10]:
# Lowercase all English words
dataset_th_df['Text_token'] = dataset_th_df['Text_token'].apply(lowercase_all)

# ungrouping the emoji’s from text
dataset_th_df['Text_token'] = dataset_th_df['Text_token'].apply(ungroup_emoji)

In [11]:
# shuffle rows in the dataset
dataset_th_df = dataset_th_df.sample(frac=1).reset_index(drop=True)

dataset_th_df.head()

,Text,Rating,Text_token
0,มาลองนำแข็งไสเกาหลี berry snow ผลไมเยอะมากรสเป...,3,"[มา, ลอง, นำ, แข็ง, ไส, เกาหลี, , berry, , s..."
1,รานนีบรรยากาศถือวานานังเลยทีเดียวแตราคาอาหารคอ...,2,"[ราน, นี, บรรยากาศ, ถือ, วา, นา, นัง, เลย, ทีเ..."
2,รานนีตังอยูในเขตของมหาลัยมหาสารคาม จอดรถไดตามข...,3,"[ราน, นี, ตัง, อ, ยู, ใน, เขต, ของ, มหาลัย, มห..."
3,รานแหลมเจริญ รานอาหารชือดังทีขึนชือเรืองอาหารท...,5,"[ราน, แหลม, เจริญ, , ราน, อาหาร, ชือ, ดัง, ที..."
4,รานนีอยูในหมูบานวิทยุการบิน ทางมาอาจจะลึกลับไป...,4,"[ราน, นีอ, ยู, ใน, หมู, บาน, วิทยุ, การ, บิน, ..."


In [12]:
# save processed dataset to file
dataset_th_df.to_csv('train_thai_processed_dataset.csv', index=False)